# Getting the list of variables from OECD.stat

In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

Replace the 'mei' in the url with the code of the database of interest. I found the dataset identifier using the 'Developer API' option in the export menu. 

* mei = Monthly Economic Indicators
* mei_trd = Monthly International Merchandise Trade

Get the contents of the url and write it to a file so that we can work offline if need be. 

In [2]:
url = 'https://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/mei'
reply = requests.get(url) 

with open('mei_structure.xml', 'wb') as f: 
        f.write(reply.content) 

Try pasting the url into a web browser. You should get back an xml "tree." This is useful to look at while you are figuring out what to extract. 

Create the xml tree in python and get the root element. 

In [9]:
tree = ET.parse('mei_structure.xml')
root = tree.getroot()

In [10]:
type(tree)

xml.etree.ElementTree.ElementTree

Now we enumerate the child elements of the root.

In [4]:
for child in root:
    print(child.tag, child.attrib)

{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message}Header {}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message}CodeLists {}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message}Concepts {}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message}KeyFamilies {}


We want to get into the CodeLists. We can reference the child like we do a list object `root[1]` gets us the second child. We can then enumerate the children of that element.  

In [5]:
for child in root[1]:
    print(child.tag, child.attrib)

{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_LOCATION', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_SUBJECT', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_MEASURE', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_FREQUENCY', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_TIME', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_OBS_STATUS', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_UNIT', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': 'CL_MEI_POWERCODE', 'agencyID': 'OECD'}
{http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure}CodeList {'id': '

We want the 'CL_MEI_SUBJECT' list. 

Now we will enumerate the subject list and take two bits of data. 

1. The 'value' which hold the code
2. The first child's text, which is the description in English

We save these data into lists. The first two elements do not have children, so we need to account for them. 

In [6]:
subjects = root[1][1]
codes, descs = [], []

for i in subjects:
    print(i.get('value'), end=' ')
    codes.append(i.get('value'))
    
    if len(i) != 0:
        print(i[0].text)
        descs.append(i[0].text)

    else:
        print('')
        descs.append('')
        
    

None 
None 
LO Leading Indicators OECD
LORS Leading Indicators OECD > Reference series
LORSGP Leading Indicators OECD > Reference series > Gross Domestic Product (GDP)
LORSGPRT Leading Indicators OECD > Reference series > Gross Domestic Product (GDP) > Ratio to trend
LORSGPNO Leading Indicators OECD > Reference series > Gross Domestic Product (GDP) > Normalised
LORSGPTD Leading Indicators OECD > Reference series > Gross Domestic Product (GDP) > Trend
LORSGPOR Leading Indicators OECD > Reference series > Gross Domestic Product (GDP) > Original series
LOLI Leading Indicators OECD > Leading indicators
LOLITO Leading Indicators OECD > Leading indicators > CLI
LOLITOAA Leading Indicators OECD > Leading indicators > CLI > Amplitude adjusted
LOLITONO Leading Indicators OECD > Leading indicators > CLI > Normalised
LOLITOTR Leading Indicators OECD > Leading indicators > CLI > Trend restored
LOCO Leading Indicators OECD > Component series
LOCOPA Leading Indicators OECD > Component series > Balan

CPHPLA Consumer Price Index > Harmonised prices > All items less food, energy, tobacco, alcohol
CPHPLA01 Consumer Price Index > Harmonised prices > All items less food, energy, tobacco, alcohol > Total
CPHP01 Consumer Price Index > Harmonised prices > Food and non-Alcoholic beverages (COICOP 01)
CPHP0100 Consumer Price Index > Harmonised prices > Food and non-Alcoholic beverages (COICOP 01) > Total
CPHP02 Consumer Price Index > Harmonised prices > Alcoholic beverages, tobacco and narcotics (COICOP 02)
CPHP0200 Consumer Price Index > Harmonised prices > Alcoholic beverages, tobacco and narcotics (COICOP 02) > Total
CPHP03 Consumer Price Index > Harmonised prices > Clothing and footwear (COICOP 03)
CPHP0300 Consumer Price Index > Harmonised prices > Clothing and footwear (COICOP 03) > Total
CPHP04 Consumer Price Index > Harmonised prices > Housing, water, electricity, gas and other fuels (COICOP 04)
CPHP0400 Consumer Price Index > Harmonised prices > Housing, water, electricity, gas and 

PWHP03 Price Weights > Harmonised Prices > Clothing and Footwear
PWHP0300 Price Weights > Harmonised Prices > Clothing and Footwear > Total
PWHP0310 Price Weights > Harmonised Prices > Clothing and Footwear > Clothing
PWHP0311 Price Weights > Harmonised Prices > Clothing and Footwear > Clothing materials
PWHP0312 Price Weights > Harmonised Prices > Clothing and Footwear > Garments
PWHP0313 Price Weights > Harmonised Prices > Clothing and Footwear > Other articles of clothing and clothing accessories
PWHP0314 Price Weights > Harmonised Prices > Clothing and Footwear > Cleaning, repair and hire of clothing
PWHP0320 Price Weights > Harmonised Prices > Clothing and Footwear > Footwear including repair
PWHP04 Price Weights > Harmonised Prices > Housing, Water, Electricity, Gas and other Fuels
PWHP0400 Price Weights > Harmonised Prices > Housing, Water, Electricity, Gas and other Fuels > Total
PWHP0410 Price Weights > Harmonised Prices > Housing, Water, Electricity, Gas and other Fuels > Act

Turn these lists into a DataFrame and write it to a csv file. 

Now we have a list of all the variables that does not require using the clunky OECD website. 

In [7]:
mei_subjects = pd.DataFrame({'codes':codes, 'description':descs})
mei_subjects = mei_subjects.iloc[2:]
mei_subjects.to_csv('mei_subjects.csv', index=False)

In [8]:
mei_subjects.head()

,codes,description
2,LO,Leading Indicators OECD
3,LORS,Leading Indicators OECD > Reference series
4,LORSGP,Leading Indicators OECD > Reference series > G...
5,LORSGPRT,Leading Indicators OECD > Reference series > G...
6,LORSGPNO,Leading Indicators OECD > Reference series > G...
